In [3]:
%run ../../data/api_keys.ipynb
%run ../modulos.ipynb
%run ../funciones_temp.ipynb

In [4]:
def request_github(url,personal_access=personal_access_token):
    headers = {"Authorization": f"Bearer {personal_access}"}

    res=requests.get(url,headers=headers)
    return res.json()

In [5]:
def get_pages_repos(min_estrellas, max_estrellas, num_pagina=1):
    return f'https://api.github.com/search/repositories?per_page=100&page={num_pagina}&q=stars%3A{min_estrellas}..{max_estrellas}&sort=forks&order=asc'

In [6]:
headers = {"Authorization": f"Bearer {personal_access_token}","Accept": "application/vnd.github.v3.star+json","content-type": "application/json"}

In [7]:
client=pymongo.MongoClient('mongodb://localhost:27017/')
database=client['promisearch'] # database
collecion_repos=database['repositorios_prueba'] # colección
len(list(collecion_repos.find()))

90

## Inserciones a DB

In [35]:
def obtener_hist(user,repo,atributo,nom_col):

    consulta_url = f"https://api.github.com/repos/{user}/{repo}/{atributo}"
    repos_resp = []
    
    page=1
    response_no_json = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page))
    response=response_no_json.json() # primeras 100
    
    # Estos try son para cuando excedes el límite de 5000 por hora 
    try:
        repos_resp.extend([re[nom_col] for re in response])
    except:
        print('Número máximo alcanzado, esperando 30s')
        time.sleep(30)
        page=page-1
        

    while len(response)!=0:
        page+=1
        response = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page)).json()
        
        try:
            repos_resp.extend([re[nom_col] for re in response])
        except:
            print('Número máximo alcanzado, esperando 30s')
            time.sleep(30)
            page=page-1
            
            

  
    serie_acum=lista_fecha_a_df(repos_resp)
    try:
        cuantiles=pd.qcut(serie_acum, q=4, retbins=True)[1]
        cuant_prop=f'{cuantiles[1]},{",".join([str(round(ai,4)) for ai in list(cuantiles[2:] / cuantiles[1:-1])])}'
    except:
        return '0,0,0,0'

    return cuant_prop

Límite de '5,000 requests per hour'

In [36]:
def obtener_info_repo(user,repo):
    """Regresa json con info histórica y del repo"""

    star_count_url = "https://api.github.com/repos/"+user+"/"+repo

    response1 = request_github_no_json(star_count_url)
     
    cuantiles_estrellas = obtener_hist(user,repo,'stargazers','starred_at')
    cuantiles_forks = obtener_hist(user,repo,'forks','created_at')

    resp_json=response1.json()
    resp_json['cuant_estrellas']=cuantiles_estrellas
    resp_json['cuant_forks']=cuantiles_forks
 
    return resp_json

In [20]:
# Prueba de que todo está funcionando
res_json=obtener_info_repo('pysentimiento','pysentimiento')
print(res_json.get('id',np.nan))
print(res_json.get('html_url',np.nan))
print(res_json.get('cuant_estrellas',np.nan))
print(res_json.get('cuant_forks',np.nan))
print(res_json.get('size',np.nan))
print(res_json.get('stargazers_count',np.nan))
print(res_json.get('forks',np.nan))
print(res_json.get('open_issues',np.nan))
print(res_json.get('subscribers_count',np.nan))
print(res_json.get('network_count',np.nan))

1
2
3
1
292896321
https://github.com/pysentimiento/pysentimiento
80.25,1.6386,1.2833,1.2741
7.5,1.7333,1.4231,1.2973
14289
215
24
4
7
24


In [23]:
def obtener_repos_de_pags(min_estrellas,max_estrellas,numero_requests,num_muestreo, k):
    repos_final = {}
    num_pagina=1 
    
    # Sólo agarrar muestreo de repos
    for _ in range(numero_requests):
        repos_bins = []

        i=0
        while i <1:
            # Construyo la URL
            url_objetivo = get_pages_repos(min_estrellas, max_estrellas, num_pagina)
            #print(num_pagina)
            print(url_objetivo)
            # Aqui hago la peticion a github
            search_github = requests.get(url_objetivo ,headers=headers)
            search_github_json = search_github.json()
            try:
                lista_repos = search_github_json['items'] # 100 repos
                lista_random = random.sample(lista_repos, num_muestreo)   
                print(f'Todo bien {k}')             
                repos_bins.extend(lista_random)
                
            except:  
                print('A dormir')
                time.sleep(61)
                i-=1
            i+=1
        num_pagina += 1
        
        repos_final[f'{_}'] = repos_bins
        collecion_repos.insert_one({f'{k}':repos_bins})
    return repos_final

In [24]:
# collecion_repos.drop()

In [68]:
# Esto es para obtener repos al azar de diferentes páginas
num_paginas_visitar=1
num_repos_por_pagina=5

#------Memo----------- 50 
# for k in range(1, 10):
# Avitua
# # for k in range(10, 1000000000):
start=1
stop=10
for k in range(start, stop):
    obtener_repos_de_pags(30*k,30*(k+1),num_paginas_visitar,num_repos_por_pagina, k)

https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A30..60&sort=forks&order=asc
Todo bien 1
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A60..90&sort=forks&order=asc
Todo bien 2
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A90..120&sort=forks&order=asc
Todo bien 3
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A120..150&sort=forks&order=asc
Todo bien 4
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A150..180&sort=forks&order=asc
Todo bien 5
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A180..210&sort=forks&order=asc
Todo bien 6
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A210..240&sort=forks&order=asc
Todo bien 7
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A240..270&sort=forks&order=asc
Todo bien 8
https://api.github.com/search/repositories?per_page=100&page=1&q=stars%3A270..300&sor

In [69]:
s=set()
for c in list(collecion_repos.find()):
  s.add(list(c.keys())[1])
  
a=[]
for si in s:
  a.append(int(si))
a=np.array(a)

a.sort()
a

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

## Creación de csv

In [70]:
id_repos = []
name_repos = []
sizes = []
stargazers_count_list = []
forks_list = []
open_issues_list = []
watchers = []
network_count_list = []
suscribers_list = []
repos1=[]

cuantiles_star=[]
cuantiles_forks=[]
lenguajes=[]

# comienzo final dice en que rango de la base de datos se van a buscar los datos de los repos
comienzo=1
final=10
for k in tqdm_notebook(range(comienzo,final)):
    lista_dict=list(collecion_repos.find({ f'{k}' : { '$exists' : 'true' } }))[0][f'{k}']
    #lista_dict=repos_final[f'{k}']
    for repo in tqdm_notebook(lista_dict, desc='Inner loop'):
        repo_name=repo['name']
        user=repo['owner']['login']
        # --------------------------------------------------
        json_repo=obtener_info_repo(user,repo_name)

        id_repos.append(json_repo.get('id',np.nan))
        name_repos.append(json_repo.get('html_url',np.nan))
        cuantiles_star.append(json_repo.get('cuant_estrellas',np.nan))
        cuantiles_forks.append(json_repo.get('cuant_forks',np.nan))
        lenguajes.append(json_repo.get('language',np.nan))
        sizes.append(json_repo.get('size',np.nan))
        stargazers_count_list.append(json_repo.get('stargazers_count',np.nan))
        forks_list.append(json_repo.get('forks',np.nan))
        open_issues_list.append(json_repo.get('open_issues',np.nan))
        watchers.append(json_repo.get('subscribers_count',np.nan))
        network_count_list.append(json_repo.get('network_count',np.nan)) # número de personas que han "contribuido"

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/9 [00:00<?, ?it/s]

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

...

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

In [71]:
repos_df = pd.DataFrame(dict(
    ids=id_repos,
    name_repo=name_repos,
    lenguaje=lenguajes,
    stargazers=stargazers_count_list,
    watchers=watchers,
    cuantiles_star=cuantiles_star,
    cuantiles_forks=cuantiles_forks,
    forks=forks_list,
    # used_by=[],   
    tamaño_kb=sizes,
    issues=open_issues_list,
    network_size=network_count_list
    # pull_request=[],
    # branches=[],
    # no_commits=[]
))

repos_df

,ids,name_repo,lenguaje,stargazers,watchers,cuantiles_star,cuantiles_forks,forks,tamaño_kb,issues,network_size
0,24815453,https://github.com/hokaccha/sprinkler,Go,50,2,"34.75,1.1367,1.1266,1.1236","0,0,0,0",0,520,0,0
1,20676037,https://github.com/rstacruz/pre.js,JavaScript,52,5,"35.25,1.1489,1.1296,1.1366","0,0,0,0",0,53,2,0
2,116265184,https://github.com/icebreaker/parsello,C,44,4,"23.75,1.3684,1.1769,1.1503","0,0,0,0",0,71,0,0
3,24593822,https://github.com/s4y/fancyfocus,JavaScript,40,4,"19.0,1.6316,1.129,1.1429","0,0,0,0",0,280,0,0
4,134693886,https://github.com/lukego/easynic,None,43,19,"20.75,1.4217,1.2288,1.1862","0,0,0,0",0,5,14,0
5,30088051,https://github.com/lutherism/GraphNoQL,JavaScript,66,9,"25.75,1.4951,1.3831,1.2394","0,0,0,0",0,224,0,0
6,57558462,https://github.com/doublesecretagency/craft-in...,PHP,66,2,"35.75,1.2727,1.2363,1.1733","0,0,0,0",0,308,4,0
7,186597250,https://github.com/SystangoTechnologies/server...,JavaScript,74,2,"7.0,5.0,2.0286,1.0423","0,0,0,0",0,9,1,0
8,148261932,https://github.com/Ao-Lee/AveragePrecision,Python,81,2,"28.75,1.7565,1.3317,1.2045","0,0,0,0",0,143,1,0
9,443606268,https://github.com/Lumince/Dolphin-MMJR2,C++,69,6,"24.0,1.625,1.4103,1.2545","0,0,0,0",0,385902,1,0


In [72]:
repos_df[['star_q1','star_gq2','star_gq3','star_gq4']]=repos_df.cuantiles_star.str.split(',',expand=True) 
repos_df.drop(columns=['cuantiles_star'],inplace=True)
repos_df[['fork_q1','fork_gq2','fork_gq3','fork_gq4']]=repos_df.cuantiles_forks.str.split(',',expand=True) 
repos_df.drop(columns=['cuantiles_forks'],inplace=True)
repos_df.tail()

,ids,name_repo,lenguaje,stargazers,watchers,forks,tamaño_kb,issues,network_size,star_q1,star_gq2,star_gq3,star_gq4,fork_q1,fork_gq2,fork_gq3,fork_gq4
40,39277038,https://github.com/whosonfirst-data/whosonfirs...,None,287,24,5,32,483,5,87.0,1.7126,1.4899,1.2928,2.0,1.5,1.3333,1.25
41,154956785,https://github.com/notomo/gesture.nvim,Lua,277,5,1,450,0,1,88.75,1.7127,1.403,1.2989,0,0,0,0
42,191300191,https://github.com/bigdongdongCLUB/newGCP,None,291,25,0,0,3,0,94.5,2.0317,1.3099,1.1571,0,0,0,0
43,275096677,https://github.com/lxgw/yozai-font,None,301,10,3,140957,2,3,65.5,2.8092,1.3315,1.2286,1.5,1.3333,1.25,1.2
44,266194816,https://github.com/junglejs/junglejs,JavaScript,277,13,5,1093,21,5,109.0,1.6606,1.2983,1.1787,2.0,1.5,1.3333,1.25


In [74]:
print(repos_df.stargazers.isna().sum())
print(repos_df.forks.isna().sum())
repos_df.stargazers.describe()

0
0


count     45.000000
mean     165.422222
std       79.515433
min       40.000000
25%       96.000000
50%      171.000000
75%      222.000000
max      301.000000
Name: stargazers, dtype: float64

In [75]:
print(comienzo*30,final*30)

30 300


In [76]:
repos_df.to_csv(f'../../data/data_sin_etiqueta_jala_{comienzo*30}-{final*30}.csv', index=False)